# 10 notebook that applies all project cards (replacing notebooks 02-09)

In [1]:
import os
import sys
import yaml
import pickle
import glob
import copy

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

# Remote I/O and parameters

In [4]:
# root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'travel-model-two-networks')
root_dir = "D:/github/lfs-cleaning/travel-model-two-networks"
input_dir = os.path.join(root_dir, 'data', 'processed', 'version_08')
output_dir = os.path.join(root_dir, 'data', 'processed', 'version_09')
card_dir = os.path.join(root_dir, 'project_cards')
# lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'mtc-Lasso')
lasso_dir = "Z:/Data/Users/Sijia/MTC/github/Lasso"

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-05-04 11:12:22, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-05-04 11:12:22, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-05-04 11:12:22, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-05-04 11:12:22, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


# Read pickle

In [6]:
version_01_pickle_file_name = os.path.join(input_dir, 'working_scenario_01.pickle')
v_01_scenario = pickle.load(open(version_01_pickle_file_name, 'rb'))

# Create scenario and apply projects

waiting on dependencies to work

In [7]:
v_01_scenario.road_net.links_df["id"] = v_01_scenario.road_net.links_df["shstGeometryId"]

In [8]:
step1_scenario = Scenario.create_scenario(
    base_scenario=v_01_scenario,
    card_directory = card_dir,
    tags = ['highway review', "Major Arterial Review", "Reversible Lanes", 'Bus Only', 'Toll Plaza'],
    validate_project_cards=False)

2021-05-04 11:13:13, INFO: Creating Scenario
2021-05-04 11:13:13, INFO: Creating Scenario


z:\data\users\sijia\mtc\github\network_wrangler\network_wrangler\projectcard.py:115: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  attribute_dictionary = yaml.load(_yaml)


In [9]:
step1_scenario.apply_all_projects()

2021-05-04 11:13:20, INFO: Applying Add San Mateo Bridge Toll Plaza
2021-05-04 11:13:20, INFO: Applying Add San Mateo Bridge Toll Plaza
2021-05-04 11:13:20, INFO: Applying Add San Mateo Bridge Toll Plaza
2021-05-04 11:13:20, INFO: Applying Add San Mateo Bridge Toll Plaza
2021-05-04 11:13:20, INFO: Applying Project to Roadway Network: Add San Mateo Bridge Toll Plaza
2021-05-04 11:13:20, INFO: Applying Project to Roadway Network: Add San Mateo Bridge Toll Plaza
2021-05-04 11:13:22, INFO: Applying Project to Transit Network: Add San Mateo Bridge Toll Plaza
2021-05-04 11:13:22, INFO: Applying Project to Transit Network: Add San Mateo Bridge Toll Plaza
2021-05-04 11:13:22, WARNING: Roadway Deletion not yet implemented in Transit; ignoring
2021-05-04 11:13:22, WARNING: Roadway Deletion not yet implemented in Transit; ignoring
2021-05-04 11:13:22, INFO: Applying Add San Mateo Bridge Toll Plaza
2021-05-04 11:13:22, INFO: Applying Add San Mateo Bridge Toll Plaza
2021-05-04 11:13:22, INFO: Apply

c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


2021-05-04 11:15:30, INFO: Applying Year 2015 Correct As Is
2021-05-04 11:15:30, INFO: Applying Year 2015 Correct As Is
2021-05-04 11:15:30, INFO: Applying Year 2015 Correct As Is
2021-05-04 11:15:30, INFO: Applying Year 2015 Correct As Is
2021-05-04 11:15:30, INFO: Applying Project to Roadway Network: Year 2015 Correct As Is
2021-05-04 11:15:30, INFO: Applying Project to Roadway Network: Year 2015 Correct As Is
2021-05-04 11:15:31, INFO: Applying Year 2015 Modify Facility Type to Service
2021-05-04 11:15:31, INFO: Applying Year 2015 Modify Facility Type to Service
2021-05-04 11:15:31, INFO: Applying Year 2015 Modify Facility Type to Service
2021-05-04 11:15:31, INFO: Applying Year 2015 Modify Facility Type to Service
2021-05-04 11:15:31, INFO: Applying Project to Roadway Network: Year 2015 Modify Facility Type to Service
2021-05-04 11:15:31, INFO: Applying Project to Roadway Network: Year 2015 Modify Facility Type to Service


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


2021-05-04 11:15:31, INFO: Applying Year 2015 Modify Facility Type to Ramp shape incorrect
2021-05-04 11:15:31, INFO: Applying Year 2015 Modify Facility Type to Ramp shape incorrect
2021-05-04 11:15:31, INFO: Applying Year 2015 Modify Facility Type to Ramp shape incorrect
2021-05-04 11:15:31, INFO: Applying Year 2015 Modify Facility Type to Ramp shape incorrect
2021-05-04 11:15:31, INFO: Applying Project to Roadway Network: Year 2015 Modify Facility Type to Ramp shape incorrect
2021-05-04 11:15:31, INFO: Applying Project to Roadway Network: Year 2015 Modify Facility Type to Ramp shape incorrect
2021-05-04 11:15:31, INFO: Applying Year 2015 Modify Facility Type to Ramp
2021-05-04 11:15:31, INFO: Applying Year 2015 Modify Facility Type to Ramp
2021-05-04 11:15:31, INFO: Applying Year 2015 Modify Facility Type to Ramp
2021-05-04 11:15:31, INFO: Applying Year 2015 Modify Facility Type to Ramp
2021-05-04 11:15:31, INFO: Applying Project to Roadway Network: Year 2015 Modify Facility Type to 

2021-05-04 11:15:36, INFO: Applying Project to Roadway Network: Year 2015 Modify Lanes to 5 Lanes
2021-05-04 11:15:37, INFO: Applying Year 2015 Should be 4 Lane should be Expressway
2021-05-04 11:15:37, INFO: Applying Year 2015 Should be 4 Lane should be Expressway
2021-05-04 11:15:37, INFO: Applying Year 2015 Should be 4 Lane should be Expressway
2021-05-04 11:15:37, INFO: Applying Year 2015 Should be 4 Lane should be Expressway
2021-05-04 11:15:37, INFO: Applying Project to Roadway Network: Year 2015 Should be 4 Lane should be Expressway
2021-05-04 11:15:37, INFO: Applying Project to Roadway Network: Year 2015 Should be 4 Lane should be Expressway
2021-05-04 11:15:37, INFO: Applying Year 2015 Should be 4 Lane should be Divided Arterial
2021-05-04 11:15:37, INFO: Applying Year 2015 Should be 4 Lane should be Divided Arterial
2021-05-04 11:15:37, INFO: Applying Year 2015 Should be 4 Lane should be Divided Arterial
2021-05-04 11:15:37, INFO: Applying Year 2015 Should be 4 Lane should be

2021-05-04 11:15:43, INFO: Applying Year 2015 Should be 3 Lane should be Undivided Arterial
2021-05-04 11:15:43, INFO: Applying Year 2015 Should be 3 Lane should be Undivided Arterial
2021-05-04 11:15:43, INFO: Applying Year 2015 Should be 3 Lane should be Undivided Arterial
2021-05-04 11:15:43, INFO: Applying Year 2015 Should be 3 Lane should be Undivided Arterial
2021-05-04 11:15:43, INFO: Applying Project to Roadway Network: Year 2015 Should be 3 Lane should be Undivided Arterial
2021-05-04 11:15:43, INFO: Applying Project to Roadway Network: Year 2015 Should be 3 Lane should be Undivided Arterial
2021-05-04 11:15:44, INFO: Applying Year 2015 Should be 2 Lane should be Undivided Arterial
2021-05-04 11:15:44, INFO: Applying Year 2015 Should be 2 Lane should be Undivided Arterial
2021-05-04 11:15:44, INFO: Applying Year 2015 Should be 2 Lane should be Undivided Arterial
2021-05-04 11:15:44, INFO: Applying Year 2015 Should be 2 Lane should be Undivided Arterial
2021-05-04 11:15:44, INF

In [10]:
step2_scenario = Scenario.create_scenario(
    base_scenario = step1_scenario,
    card_directory = card_dir,
    project_cards_list = [],
    tags = ['Managed Lanes', 'toll review', 'Exclude Trucks'],
    validate_project_cards=False)

2021-05-04 11:18:06, INFO: Creating Scenario
2021-05-04 11:18:06, INFO: Creating Scenario


In [11]:
step2_scenario.apply_all_projects()

2021-05-04 11:18:12, ERROR: Projects Year 2015 Modify toll links on Golden Gate Bridge Toll Plaza on US101 SB - Bridge Pass has A as prerequisite project which is missing for the scenario
2021-05-04 11:18:12, ERROR: Projects Year 2015 Modify toll links on Golden Gate Bridge Toll Plaza on US101 SB - Bridge Pass has A as prerequisite project which is missing for the scenario
2021-05-04 11:18:12, ERROR: Projects Year 2015 Modify toll links on Golden Gate Bridge Toll Plaza on US101 SB - Bridge Pass has d as prerequisite project which is missing for the scenario
2021-05-04 11:18:12, ERROR: Projects Year 2015 Modify toll links on Golden Gate Bridge Toll Plaza on US101 SB - Bridge Pass has d as prerequisite project which is missing for the scenario
2021-05-04 11:18:12, ERROR: Projects Year 2015 Modify toll links on Golden Gate Bridge Toll Plaza on US101 SB - Bridge Pass has d as prerequisite project which is missing for the scenario
2021-05-04 11:18:12, ERROR: Projects Year 2015 Modify toll l

2021-05-04 11:18:12, ERROR: Projects Year 2015 Modify toll links on Golden Gate Bridge Toll Plaza on US101 SB - Bridge Pass has   as prerequisite project which is missing for the scenario
2021-05-04 11:18:12, ERROR: Projects Year 2015 Modify toll links on Golden Gate Bridge Toll Plaza on US101 SB - Bridge Pass has   as prerequisite project which is missing for the scenario
2021-05-04 11:18:12, ERROR: Projects Year 2015 Modify toll links on Golden Gate Bridge Toll Plaza on US101 SB - Bridge Pass has T as prerequisite project which is missing for the scenario
2021-05-04 11:18:12, ERROR: Projects Year 2015 Modify toll links on Golden Gate Bridge Toll Plaza on US101 SB - Bridge Pass has T as prerequisite project which is missing for the scenario
2021-05-04 11:18:12, ERROR: Projects Year 2015 Modify toll links on Golden Gate Bridge Toll Plaza on US101 SB - Bridge Pass has o as prerequisite project which is missing for the scenario
2021-05-04 11:18:12, ERROR: Projects Year 2015 Modify toll l

2021-05-04 11:18:12, ERROR: Projects Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass has t as prerequisite project which is missing for the scenario
2021-05-04 11:18:12, ERROR: Projects Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass has t as prerequisite project which is missing for the scenario
2021-05-04 11:18:12, ERROR: Projects Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass has e as prerequisite project which is missing for the scenario
2021-05-04 11:18:12, ERROR: Projects Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass has e as prerequisite project which is missing for the scenario
2021-05-04 11:18:12, ERROR: Projects Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass has o as prerequisite project which is missi

2021-05-04 11:18:12, ERROR: Projects Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass has a as prerequisite project which is missing for the scenario
2021-05-04 11:18:12, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2021-05-04 11:18:12, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2021-05-04 11:18:12, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2021-05-04 11:18:12, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2021-05-04 11:18:12, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2021-05-04 11:18:12, INFO: Applying Project to Roadway Network: Year 2015

2021-05-04 11:19:18, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on US-101 NB from North of Petaluma to South of Healdsburg
2021-05-04 11:19:32, INFO: Applying Year 2015 Modify HOV Lanes and USE on US 101 NB Cochran Road to CA-85
2021-05-04 11:19:32, INFO: Applying Year 2015 Modify HOV Lanes and USE on US 101 NB Cochran Road to CA-85
2021-05-04 11:19:32, INFO: Applying Year 2015 Modify HOV Lanes and USE on US 101 NB Cochran Road to CA-85
2021-05-04 11:19:32, INFO: Applying Year 2015 Modify HOV Lanes and USE on US 101 NB Cochran Road to CA-85
2021-05-04 11:19:32, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on US 101 NB Cochran Road to CA-85
2021-05-04 11:19:32, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on US 101 NB Cochran Road to CA-85
2021-05-04 11:19:55, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Tomas Expressway SB I-880 to Hwy 17
2021-05-04 11:19:55, INFO: Applying 

2021-05-04 11:22:18, INFO: Applying Year 2015 Modify Express Lanes and USE on I-880 NB CA-237 to Dixon Road
2021-05-04 11:22:18, INFO: Applying Year 2015 Modify Express Lanes and USE on I-880 NB CA-237 to Dixon Road
2021-05-04 11:22:18, INFO: Applying Project to Roadway Network: Year 2015 Modify Express Lanes and USE on I-880 NB CA-237 to Dixon Road
2021-05-04 11:22:18, INFO: Applying Project to Roadway Network: Year 2015 Modify Express Lanes and USE on I-880 NB CA-237 to Dixon Road
2021-05-04 11:22:19, INFO: Applying Year 2015 Modify USE on I-80 WB to Bay Bridge Toll Plaza
2021-05-04 11:22:19, INFO: Applying Year 2015 Modify USE on I-80 WB to Bay Bridge Toll Plaza
2021-05-04 11:22:19, INFO: Applying Year 2015 Modify USE on I-80 WB to Bay Bridge Toll Plaza
2021-05-04 11:22:19, INFO: Applying Year 2015 Modify USE on I-80 WB to Bay Bridge Toll Plaza
2021-05-04 11:22:19, INFO: Applying Project to Roadway Network: Year 2015 Modify USE on I-80 WB to Bay Bridge Toll Plaza
2021-05-04 11:22:19

2021-05-04 11:23:19, INFO: Applying Year 2015 Modify HOV Lanes USE on I-680 NB CA-242 to Benicia Bridge
2021-05-04 11:23:19, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes USE on I-680 NB CA-242 to Benicia Bridge
2021-05-04 11:23:19, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes USE on I-680 NB CA-242 to Benicia Bridge
2021-05-04 11:23:22, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-680 NB from Acosta Blvd to Rudgear Road
2021-05-04 11:23:22, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-680 NB from Acosta Blvd to Rudgear Road
2021-05-04 11:23:22, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-680 NB from Acosta Blvd to Rudgear Road
2021-05-04 11:23:22, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-680 NB from Acosta Blvd to Rudgear Road
2021-05-04 11:23:22, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on I-680 NB from Acosta Blvd to Rudgear Road
2021-05-04 11:23:22, INF

2021-05-04 11:23:43, INFO: Applying Project to Roadway Network: Year 2015 Modify Express Lanes and USE on I-580 EB (segment 01) Hacienda Drive to Airway_Blvd
2021-05-04 11:23:45, INFO: Applying Year 2015 Modify Express Lanes and USE on I-580 EB Airway Blvd to Livermore Ave
2021-05-04 11:23:45, INFO: Applying Year 2015 Modify Express Lanes and USE on I-580 EB Airway Blvd to Livermore Ave
2021-05-04 11:23:45, INFO: Applying Year 2015 Modify Express Lanes and USE on I-580 EB Airway Blvd to Livermore Ave
2021-05-04 11:23:45, INFO: Applying Year 2015 Modify Express Lanes and USE on I-580 EB Airway Blvd to Livermore Ave
2021-05-04 11:23:45, INFO: Applying Project to Roadway Network: Year 2015 Modify Express Lanes and USE on I-580 EB Airway Blvd to Livermore Ave
2021-05-04 11:23:45, INFO: Applying Project to Roadway Network: Year 2015 Modify Express Lanes and USE on I-580 EB Airway Blvd to Livermore Ave
2021-05-04 11:23:46, INFO: Applying Year 2015 Modify HOV lanes and USE on Hwy 4 WB from De

2021-05-04 11:25:22, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-280 SB Magdalena Ave to Southwest Expressway
2021-05-04 11:25:22, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-280 SB Magdalena Ave to Southwest Expressway
2021-05-04 11:25:22, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-280 SB Magdalena Ave to Southwest Expressway
2021-05-04 11:25:22, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-280 SB Magdalena Ave to Southwest Expressway
2021-05-04 11:25:22, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on CA-280 SB Magdalena Ave to Southwest Expressway
2021-05-04 11:25:22, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on CA-280 SB Magdalena Ave to Southwest Expressway
2021-05-04 11:25:27, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-280 NB Bascom Ave to Magdalena Ave
2021-05-04 11:25:27, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-280 NB Bascom Ave to Ma

2021-05-04 11:25:44, INFO: Applying Year 2015 Modify HOV Lanes-Fastrak Lanes and USE on San Francisco-Oakland Bay Bridge WB Toll Plaza - Bridge Pass
2021-05-04 11:25:44, INFO: Applying Year 2015 Modify HOV Lanes-Fastrak Lanes and USE on San Francisco-Oakland Bay Bridge WB Toll Plaza - Bridge Pass
2021-05-04 11:25:44, INFO: Applying Year 2015 Modify HOV Lanes-Fastrak Lanes and USE on San Francisco-Oakland Bay Bridge WB Toll Plaza - Bridge Pass
2021-05-04 11:25:44, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes-Fastrak Lanes and USE on San Francisco-Oakland Bay Bridge WB Toll Plaza - Bridge Pass
2021-05-04 11:25:44, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes-Fastrak Lanes and USE on San Francisco-Oakland Bay Bridge WB Toll Plaza - Bridge Pass
2021-05-04 11:25:44, INFO: Applying Year 2015 Modify toll links on Golden Gate Bridge Toll Plaza on US101 SB - Bridge Pass
2021-05-04 11:25:44, INFO: Applying Year 2015 Modify toll links on Golden Ga

In [12]:
step2_scenario.road_net.links_df.shape

(1634771, 44)

In [13]:
step2_scenario.road_net.links_df.model_link_id.nunique()

1634771

In [14]:
step2_scenario.road_net.links_df.county.isnull().sum()

0

In [15]:
step2_scenario.road_net.links_df.model_link_id.max()

8063199

In [16]:
step3_scenario = Scenario.create_scenario(
    base_scenario= step2_scenario,
    card_directory = card_dir,
    project_cards_list = [],
    tags = ["Major Transit links"],
    validate_project_cards=False)

step3_scenario.apply_all_projects()

2021-05-04 11:25:48, INFO: Creating Scenario
2021-05-04 11:25:48, INFO: Creating Scenario
2021-05-04 11:25:54, INFO: Applying move street rail to street connector links
2021-05-04 11:25:54, INFO: Applying move street rail to street connector links
2021-05-04 11:25:54, INFO: Applying move street rail to street connector links
2021-05-04 11:25:54, INFO: Applying move street rail to street connector links
2021-05-04 11:25:54, INFO: Applying Project to Roadway Network: move street rail to street connector links
2021-05-04 11:25:54, INFO: Applying Project to Roadway Network: move street rail to street connector links
2021-05-04 11:25:54, INFO: Applying move street rail to street connector links
2021-05-04 11:25:54, INFO: Applying move street rail to street connector links
2021-05-04 11:25:54, INFO: Applying Project to Roadway Network: move street rail to street connector links
2021-05-04 11:25:54, INFO: Applying Project to Roadway Network: move street rail to street connector links
2021-05-

In [17]:
step3_scenario.road_net.links_df.shape

(1634789, 44)

In [18]:
step3_scenario.transit_net.set_roadnet(step3_scenario.road_net, 
                                      validate_consistency = False)

step4_scenario = Scenario.create_scenario(
    base_scenario= step3_scenario,
    card_directory = card_dir,
    project_cards_list = [],
    tags = ["Minor Transit", "Add Transit", "Major Transit", "Toll Plaza Transit"],
    validate_project_cards=False)

step4_scenario.apply_all_projects()

2021-05-04 12:05:16, INFO: Creating Scenario
2021-05-04 12:05:16, INFO: Creating Scenario
2021-05-04 12:08:53, ERROR: Projects SFMTA Route N-JUDAH has move street rail to street connector links as prerequisite project which is missing for the scenario
2021-05-04 12:08:53, ERROR: Projects SFMTA Route N-JUDAH has move street rail to street connector links as prerequisite project which is missing for the scenario
2021-05-04 12:08:53, ERROR: Projects update routing for Golden Gate Bridge Toll Plaza transit has Add Golden Gate Bridge Toll Plaza as prerequisite project which is missing for the scenario
2021-05-04 12:08:53, ERROR: Projects update routing for Golden Gate Bridge Toll Plaza transit has Add Golden Gate Bridge Toll Plaza as prerequisite project which is missing for the scenario
2021-05-04 12:08:53, ERROR: Projects update routing for San Mateo Bridge Toll Plaza transit has Add San Mateo Bridge Toll Plaza as prerequisite project which is missing for the scenario
2021-05-04 12:08:53,

c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


2021-05-04 12:09:31, INFO: Creating a new stop in stops.txt for node ID: 2146003
2021-05-04 12:09:31, INFO: Creating a new stop in stops.txt for node ID: 2064421
2021-05-04 12:09:31, INFO: Creating a new stop in stops.txt for node ID: 2064421
2021-05-04 12:09:32, INFO: Creating a new stop in stops.txt for node ID: 2078170
2021-05-04 12:09:32, INFO: Creating a new stop in stops.txt for node ID: 2078170
2021-05-04 12:09:32, INFO: Creating a new stop in stops.txt for node ID: 2104183
2021-05-04 12:09:32, INFO: Creating a new stop in stops.txt for node ID: 2104183
2021-05-04 12:09:32, INFO: Creating a new stop in stops.txt for node ID: 2015328
2021-05-04 12:09:32, INFO: Creating a new stop in stops.txt for node ID: 2015328
2021-05-04 12:09:33, INFO: Applying VTA Route 22
2021-05-04 12:09:33, INFO: Applying VTA Route 22
2021-05-04 12:09:33, INFO: Applying Project to Transit Network: VTA Route 22
2021-05-04 12:09:33, INFO: Applying Project to Transit Network: VTA Route 22
2021-05-04 12:09:35

2021-05-04 12:09:39, INFO: Creating a new stop in stops.txt for node ID: 2108685
2021-05-04 12:09:39, INFO: Creating a new stop in stops.txt for node ID: 2108685
2021-05-04 12:09:39, INFO: Creating a new stop in stops.txt for node ID: 2046425
2021-05-04 12:09:39, INFO: Creating a new stop in stops.txt for node ID: 2046425
2021-05-04 12:09:39, INFO: Creating a new stop in stops.txt for node ID: 2098451
2021-05-04 12:09:39, INFO: Creating a new stop in stops.txt for node ID: 2098451
2021-05-04 12:09:39, INFO: Creating a new stop in stops.txt for node ID: 2020562
2021-05-04 12:09:39, INFO: Creating a new stop in stops.txt for node ID: 2020562
2021-05-04 12:09:40, INFO: Creating a new stop in stops.txt for node ID: 2090983
2021-05-04 12:09:40, INFO: Creating a new stop in stops.txt for node ID: 2090983
2021-05-04 12:09:40, INFO: Creating a new stop in stops.txt for node ID: 2054217
2021-05-04 12:09:40, INFO: Creating a new stop in stops.txt for node ID: 2054217
2021-05-04 12:09:40, INFO: C

2021-05-04 12:09:49, INFO: Creating a new stop in stops.txt for node ID: 2075183
2021-05-04 12:09:49, INFO: Creating a new stop in stops.txt for node ID: 2075183
2021-05-04 12:09:49, INFO: Creating a new stop in stops.txt for node ID: 2002834
2021-05-04 12:09:49, INFO: Creating a new stop in stops.txt for node ID: 2002834
2021-05-04 12:09:49, INFO: Creating a new stop in stops.txt for node ID: 2001551
2021-05-04 12:09:49, INFO: Creating a new stop in stops.txt for node ID: 2001551
2021-05-04 12:09:49, INFO: Creating a new stop in stops.txt for node ID: 2048716
2021-05-04 12:09:49, INFO: Creating a new stop in stops.txt for node ID: 2048716
2021-05-04 12:09:49, INFO: Creating a new stop in stops.txt for node ID: 2005001
2021-05-04 12:09:49, INFO: Creating a new stop in stops.txt for node ID: 2005001
2021-05-04 12:09:49, INFO: Creating a new stop in stops.txt for node ID: 2111886
2021-05-04 12:09:49, INFO: Creating a new stop in stops.txt for node ID: 2111886
2021-05-04 12:09:49, INFO: C

...

2021-05-04 12:12:50, INFO: Creating a new stop in stops.txt for node ID: 1015387
2021-05-04 12:12:50, INFO: Creating a new stop in stops.txt for node ID: 1017471
2021-05-04 12:12:50, INFO: Creating a new stop in stops.txt for node ID: 1017471
2021-05-04 12:12:52, INFO: Applying SFMTA Route 24
2021-05-04 12:12:52, INFO: Applying SFMTA Route 24
2021-05-04 12:12:52, INFO: Applying SFMTA Route 24
2021-05-04 12:12:52, INFO: Applying SFMTA Route 24
2021-05-04 12:12:52, INFO: Applying Project to Transit Network: SFMTA Route 24
2021-05-04 12:12:52, INFO: Applying Project to Transit Network: SFMTA Route 24
2021-05-04 12:12:54, INFO: Creating a new stop in stops.txt for node ID: 1004132
2021-05-04 12:12:54, INFO: Creating a new stop in stops.txt for node ID: 1004132
2021-05-04 12:12:56, INFO: Applying SFMTA Route 24
2021-05-04 12:12:56, INFO: Applying SFMTA Route 24
2021-05-04 12:12:56, INFO: Applying Project to Transit Network: SFMTA Route 24
2021-05-04 12:12:56, INFO: Applying Project to Trans

2021-05-04 12:13:45, INFO: Creating a new stop in stops.txt for node ID: 1018498
2021-05-04 12:13:45, INFO: Creating a new stop in stops.txt for node ID: 1018498
2021-05-04 12:13:45, INFO: Creating a new stop in stops.txt for node ID: 1022481
2021-05-04 12:13:45, INFO: Creating a new stop in stops.txt for node ID: 1022481
2021-05-04 12:13:45, INFO: Applying SFMTA Route 346 Edits
2021-05-04 12:13:45, INFO: Applying SFMTA Route 346 Edits
2021-05-04 12:13:45, INFO: Applying Project to Transit Network: SFMTA Route 346 Edits
2021-05-04 12:13:45, INFO: Applying Project to Transit Network: SFMTA Route 346 Edits
2021-05-04 12:13:47, INFO: Creating a new stop in stops.txt for node ID: 1021719
2021-05-04 12:13:47, INFO: Creating a new stop in stops.txt for node ID: 1021719
2021-05-04 12:13:47, INFO: Creating a new stop in stops.txt for node ID: 1007696
2021-05-04 12:13:47, INFO: Creating a new stop in stops.txt for node ID: 1007696
2021-05-04 12:13:48, INFO: Applying SFMTA Route 346 Edits
2021-0

2021-05-04 12:14:35, INFO: Applying SFMTA Route 306 Edits
2021-05-04 12:14:35, INFO: Applying Project to Transit Network: SFMTA Route 306 Edits
2021-05-04 12:14:35, INFO: Applying Project to Transit Network: SFMTA Route 306 Edits
2021-05-04 12:14:39, INFO: Applying SFMTA Route 306 Edits
2021-05-04 12:14:39, INFO: Applying SFMTA Route 306 Edits
2021-05-04 12:14:39, INFO: Applying Project to Transit Network: SFMTA Route 306 Edits
2021-05-04 12:14:39, INFO: Applying Project to Transit Network: SFMTA Route 306 Edits
2021-05-04 12:14:41, INFO: Applying SFMTA Route 305 Edits
2021-05-04 12:14:41, INFO: Applying SFMTA Route 305 Edits
2021-05-04 12:14:41, INFO: Applying SFMTA Route 305 Edits
2021-05-04 12:14:41, INFO: Applying SFMTA Route 305 Edits
2021-05-04 12:14:41, INFO: Applying Project to Transit Network: SFMTA Route 305 Edits
2021-05-04 12:14:41, INFO: Applying Project to Transit Network: SFMTA Route 305 Edits
2021-05-04 12:14:44, INFO: Applying SFMTA Route 305 Edits
2021-05-04 12:14:44,

2021-05-04 12:30:28, INFO: Creating a new stop in stops.txt for node ID: 1513122
2021-05-04 12:30:28, INFO: Creating a new stop in stops.txt for node ID: 1513122
2021-05-04 12:30:28, INFO: Creating a new stop in stops.txt for node ID: 1514054
2021-05-04 12:30:28, INFO: Creating a new stop in stops.txt for node ID: 1514054
2021-05-04 12:30:28, INFO: Creating a new stop in stops.txt for node ID: 1543182
2021-05-04 12:30:28, INFO: Creating a new stop in stops.txt for node ID: 1543182
2021-05-04 12:30:28, INFO: Creating a new stop in stops.txt for node ID: 1512385
2021-05-04 12:30:28, INFO: Creating a new stop in stops.txt for node ID: 1512385
2021-05-04 12:30:28, INFO: Creating a new stop in stops.txt for node ID: 1534620
2021-05-04 12:30:28, INFO: Creating a new stop in stops.txt for node ID: 1534620
2021-05-04 12:30:28, INFO: Creating a new stop in stops.txt for node ID: 1503246
2021-05-04 12:30:28, INFO: Creating a new stop in stops.txt for node ID: 1503246
2021-05-04 12:30:28, INFO: C

2021-05-04 12:30:51, INFO: Applying Project to Transit Network: AC Transit Route 1
2021-05-04 12:30:53, WARNING: Missing connections from node 2592075 to node 2596647 for the new routing, find complete path using default graph
2021-05-04 12:30:53, WARNING: Missing connections from node 2592075 to node 2596647 for the new routing, find complete path using default graph
2021-05-04 12:30:53, INFO: Routing path from node 2592075 to node 2596647 for missing connections: ['2592075', '2570072', '2500672', '2505644', '2514527', '2576504', '2596647'].
2021-05-04 12:30:53, INFO: Routing path from node 2592075 to node 2596647 for missing connections: ['2592075', '2570072', '2500672', '2505644', '2514527', '2576504', '2596647'].
2021-05-04 12:30:53, WARNING: Missing connections from node 2600447 to node 2530653 for the new routing, find complete path using default graph
2021-05-04 12:30:53, WARNING: Missing connections from node 2600447 to node 2530653 for the new routing, find complete path using

In [19]:
step4_scenario.road_net = mtc.add_centroid_and_centroid_connector(
    roadway_network = step4_scenario.road_net,
    parameters = parameters)

2021-05-04 12:31:15, INFO: Adding centroid and centroid connector to standard network
2021-05-04 12:31:15, INFO: Adding centroid and centroid connector to standard network
2021-05-04 12:31:15, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-05-04 12:31:15, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-05-04 12:31:15, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-05-04 12:31:15, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-05-04 12:31:35, INFO: Finished adding centroid and centroid connectors
2021-05-04 12:31:35, INFO: Finished adding centroid and centroid connectors


In [20]:
print(step4_scenario.road_net.links_df.shape)
print(step4_scenario.road_net.links_df.model_link_id.nunique())
print(step4_scenario.road_net.nodes_df.shape)
print(step4_scenario.road_net.nodes_df.model_node_id.nunique())
print(step4_scenario.road_net.shapes_df.shape)

(2199291, 44)
2199291
(688959, 12)
688959
(1152245, 7)


In [21]:
step5_scenario = Scenario.create_scenario(
    base_scenario= step4_scenario,
    card_directory = card_dir,
    project_cards_list = [],
    tags = ['External Stations Review'],
    validate_project_cards=False)

step5_scenario.apply_all_projects()

2021-05-04 12:31:36, INFO: Creating Scenario
2021-05-04 12:31:36, INFO: Creating Scenario


z:\data\users\sijia\mtc\github\network_wrangler\network_wrangler\projectcard.py:115: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  attribute_dictionary = yaml.load(_yaml)


2021-05-04 12:35:42, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-05-04 12:35:42, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-05-04 12:35:42, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-05-04 12:35:42, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-05-04 12:35:42, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB
2021-05-04 12:35:42, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 SB


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


2021-05-04 12:38:14, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-05-04 12:38:14, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-05-04 12:38:14, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-05-04 12:38:14, INFO: Applying Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-05-04 12:38:14, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-05-04 12:38:14, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting William Elton Brown Freeway I-580 NB
2021-05-04 12:40:44, INFO: Applying Year 2015 Add centroid connector at External Station connecting Vic Fazio Highway CA-1

2021-05-04 13:03:50, INFO: Applying Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-05-04 13:03:50, INFO: Applying Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-05-04 13:03:50, INFO: Applying Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-05-04 13:03:50, INFO: Applying Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-05-04 13:03:50, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-05-04 13:03:50, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Redwood Highway US-101 NB
2021-05-04 13:06:23, INFO: Applying Year 2015 Add centroid connector at External Station connecting Pacheco Pass Highway CA-156 SB
2021-05-04 13:06:23, INFO: Applying Year 2015 Add centroi

2021-05-04 13:29:20, INFO: Applying Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 EB
2021-05-04 13:29:20, INFO: Applying Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 EB
2021-05-04 13:29:20, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 EB
2021-05-04 13:29:20, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Dwight D Eisenhower Highway I-80 EB
2021-05-04 13:31:54, INFO: Applying Year 2015 Add centroid connector at External Station connecting Cabrillo Highway CA-1 SB
2021-05-04 13:31:54, INFO: Applying Year 2015 Add centroid connector at External Station connecting Cabrillo Highway CA-1 SB
2021-05-04 13:31:54, INFO: Applying Year 2015 Add centroid connector at External Station connecting Cabrillo Highway CA-1 SB
2021-05-04 13:31:54, INFO

In [22]:
print(step5_scenario.road_net.links_df.shape)
print(step5_scenario.road_net.links_df.model_link_id.nunique())
print(step5_scenario.road_net.nodes_df.shape)
print(step5_scenario.road_net.nodes_df.model_node_id.nunique())

(2199317, 44)
2199317
(688959, 12)
688959


# sanity check

In [23]:
step5_scenario.road_net.links_df[step5_scenario.road_net.links_df.model_link_id == 3150085] # should be ramp lanes 2

,access,bike_access,drive_access,fromIntersectionId,lanes,maxspeed,name,oneWay,ref,roadway,...,transit_access,managed,ML_lanes,segment_id,ML_tollbooth,ML_useclass,ML_access,ML_egress,ML_tollseg,tollseg
765902,,False,True,d1b76ff92ba7a2915de8efc97c782d73,2,,,True,,motorway_link,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
step5_scenario.road_net.links_df[step5_scenario.road_net.links_df.model_link_id == 3107408] # bay bridge toll link

,access,bike_access,drive_access,fromIntersectionId,lanes,maxspeed,name,oneWay,ref,roadway,...,transit_access,managed,ML_lanes,segment_id,ML_tollbooth,ML_useclass,ML_access,ML_egress,ML_tollseg,tollseg
548523,"['nan', 'nan']",False,True,826a99158fd58e5483d1085c9e0d65b3,"{'default': 4, 'timeofday': [{'time': (21600, ...","['nan', 'nan']","['nan', 'nan']","[True, True]","['I 80', 'I 80']",motorway,...,NaN,1,"{'default': 0, 'timeofday': [{'time': (21600, ...",5,5,"{'default': 0, 'timeofday': [{'time': (21600, ...",all,all,NaN,NaN


In [25]:
step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 26067343
].iloc[0][["A", "B"]]

A     900020
B    2192724
Name: 2199297, dtype: object

In [26]:
# this link is splitted into two for toll plaza at GGB

step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 8016300]

,access,bike_access,drive_access,fromIntersectionId,lanes,maxspeed,name,oneWay,ref,roadway,...,transit_access,managed,ML_lanes,segment_id,ML_tollbooth,ML_useclass,ML_access,ML_egress,ML_tollseg,tollseg


In [27]:
# golden gate bridge

step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 8063198].lanes.iloc[0]

{'default': 3,
 'timeofday': [{'time': (21600, 36000), 'value': 3},
  {'time': (54000, 68400), 'value': 2}]}

In [28]:
# golden gate bridge HOV

step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 8063198].ML_lanes.iloc[0]

{'default': 0,
 'timeofday': [{'time': (21600, 36000), 'value': 1},
  {'time': (54000, 68400), 'value': 0}]}

In [29]:
# check if new node is created for toll plaza at GGB

step5_scenario.road_net.nodes_df[step5_scenario.road_net.nodes_df.model_node_id == 5030001]

,osm_node_id,shst_node_id,county,drive_access,walk_access,bike_access,model_node_id,rail_only,X,Y,geometry,farezone
644476,NaN,,Marin,1,0,0,5030001,0,-122.475698,37.808167,POINT (-122.47570 37.80817),


In [30]:
# golden gate bridge toll plaza

step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.model_link_id == 8063199].lanes.iloc[0]

9

In [31]:
step5_scenario.transit_net.feed.shapes.shape_id.nunique()

1561

In [32]:
step5_scenario.transit_net.feed.shapes[
    step5_scenario.transit_net.feed.shapes.shape_id.isin(
        ["1169", "1168", "939", "940", "943", "944"]
    )
].groupby(["shape_id"]).count()

,shape_pt_sequence,shape_osm_node_id,shape_shst_node_id,shape_model_node_id,shape_pt_lat,shape_pt_lon
shape_id,,,,,,
1168,177,108,108,177,0,0
1169,176,65,65,176,0,0
939,26,22,22,26,0,0
940,161,157,157,161,0,0
943,158,154,154,158,0,0
944,130,128,128,130,0,0


In [33]:
step5_scenario.road_net.links_df[
    step5_scenario.road_net.links_df.A.isin([2579949, 2607569])][["A", "B", "roadway","assignable"]]

,A,B,roadway,assignable
514595,2579949,2535246,residential,0
842601,2579949,2604856,residential,1
846406,2607569,2586861,residential,1
1009927,2607569,2546185,residential,1
1077889,2607569,2541364,residential,0
1645008,2579949,300963,taz,1
1645926,2607569,300963,taz,1
1820100,2579949,318103,maz,1
1820101,2579949,327248,maz,1
1820102,2579949,328044,maz,1


## Make Travel Model Networks

In [72]:
model_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = step5_scenario.road_net, 
    parameters = parameters)

2021-05-04 17:36:02, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-05-04 17:36:02, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-05-04 17:36:02, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-05-04 17:36:02, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-05-04 17:36:02, INFO: Filling nan for network from network wrangler
2021-05-04 17:36:02, INFO: Filling nan for network from network wrangler
2021-05-04 17:36:40, INFO: Converting variable type to mtc standard
2021-05-04 17:36:40, INFO: Converting variable type to mtc standard


In [73]:
print(model_net.links_df.shape)
print(model_net.links_df.model_link_id.nunique())
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

(2199317, 44)
2199317
(688959, 12)
(1152271, 7)


In [74]:
model_net = mtc.add_tap_and_tap_connector(
    roadway_network = model_net,
    transit_network = step5_scenario.transit_net, 
    parameters = parameters)

2021-05-04 17:38:04, INFO: Adding tap and tap connector to standard network
2021-05-04 17:38:04, INFO: Adding tap and tap connector to standard network
2021-05-04 17:38:04, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-05-04 17:38:04, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-05-04 17:38:04, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-05-04 17:38:04, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-05-04 17:38:05, INFO: There are 491 stops not connected to taps, connecting them to the closest taps.
2021-05-04 17:38:05, INFO: There are 491 stops not connected to taps, connecting them to the closest taps.
snapping gtfs stops to roadway node osmid...


z:\data\users\sijia\mtc\github\lasso\lasso\mtc.py:2500: FutureWarning: CRS mismatch between CRS of the passed geometries and 'crs'. Use 'GeoDataFrame.set_crs(crs, allow_override=True)' to overwrite CRS or 'GeoDataFrame.to_crs(crs)' to reproject geometries. CRS mismatch will raise an error in the future versions of GeoPandas.
  crs = "EPSG:4326"


Index(['stop_name', 'stop_desc', 'stop_lat', 'stop_lon', 'zone_id', 'stop_url',
       'location_type', 'parent_station', 'stop_timezone',
       'wheelchair_boarding', 'stop_code', 'platform_code', 'position',
       'direction', 'stop_id', 'osm_node_id', 'shst_node_id', 'A', 'tap_id',
       'index_right', 'county', 'tap_node_county_start', 'X', 'Y', 'tap_X',
       'tap_Y', 'id', 'shstGeometryId', 'geometry', 'B'],
      dtype='object')
2021-05-04 17:38:16, INFO: Finished adding tap and tap connectors
2021-05-04 17:38:16, INFO: Finished adding tap and tap connectors


In [75]:
# Vallejo Ferry to TAP

model_net.links_df[model_net.links_df.B == 3541176][["A", "roadway", "county", "rail_only", "model_link_id"]]

,A,roadway,county,rail_only,model_link_id
191843,3512789,footway,Solano,0.0,5013621
1634790,1028038,,,1.0,26067374
1634792,1028039,,,1.0,26067376
2235475,590188,tap,Solano,NaN,5157453


In [76]:
print(model_net.links_df.shape)
print(model_net.links_df.model_link_id.nunique())
print(model_net.nodes_df.shape)
print(model_net.nodes_df.model_node_id.nunique())
print(model_net.shapes_df.shape)
print(model_net.shapes_df.id.nunique())

(2235541, 44)
2235541
(694959, 12)
694959
(1170383, 7)
1170383


In [77]:
model_net = mtc.roadway_standard_to_mtc_network(model_net, parameters)

2021-05-04 17:38:36, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2021-05-04 17:38:36, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2021-05-04 17:38:36, INFO: Creating managed lane network.
2021-05-04 17:38:36, INFO: Creating managed lane network.
2021-05-04 17:38:36, INFO: Creating network with duplicated managed lanes
2021-05-04 17:38:36, INFO: Creating network with duplicated managed lanes


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
c:\users\wang

2021-05-04 18:01:31, INFO: Determining cntype
2021-05-04 18:01:31, INFO: Determining cntype
2021-05-04 18:01:31, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-05-04 18:01:31, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-05-04 18:01:31, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-05-04 18:01:31, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-05-04 18:01:31, INFO: Calculating and adding roadway network variable: cntype
2021-05-04 18:01:31, INFO: Calculating and adding roadway network variable: cntype
2021-05-04 18:03:07, INFO: Finished determining variable: cntype
2021-05-04 18:03:07, INFO: Finished determining variable: cntype
2021-05-04 18:03:07, INFO: Determining transit
2021-05-04 18:03:07, INFO: Determining transit
2021-05-04 18:03:07, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-05-04 18:03:07, INFO: Lasso base directory set a

In [78]:
print(model_net.links_df.shape)
print(model_net.links_df.model_link_id.nunique())
print(model_net.nodes_df.shape)
print(model_net.nodes_df.model_node_id.nunique())
print(model_net.shapes_df.shape)
print(model_net.shapes_df.id.nunique())

(2239940, 55)
2239940
(696467, 13)
696467
(1174793, 8)
1174793


In [79]:
model_net.links_df.rail_only.value_counts()

0    2239175
1        765
Name: rail_only, dtype: int64

In [80]:
model_net.links_df[(model_net.links_df.bus_only == 1)].cntype.unique()

array(['TANA'], dtype=object)

In [81]:
model_net.links_df.assignable.value_counts()

0    1282706
1     957234
Name: assignable, dtype: int64

In [82]:
model_net.links_df[model_net.links_df.cntype == "TAZ"].assignable.value_counts()

1    32986
Name: assignable, dtype: int64

In [83]:
model_net.links_df[model_net.links_df.cntype == "MAZ"].assignable.value_counts()

1    531534
Name: assignable, dtype: int64

In [87]:
model_net.links_df[model_net.links_df.A == 2601773].roadway

1284           motorway
4220          ml_access
803691    motorway_link
Name: roadway, dtype: object

# Write to Disk

In [88]:
model_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'links.shp'),
    output_node_shp = os.path.join(output_dir, 'nodes.shp'),
    link_output_variables = ["model_link_id", "A", "B", "geometry", "cntype", "lanes_AM", "assignable"],
    node_output_variables = ["model_node_id", "geometry"],
    data_to_csv = False,
    data_to_dbf = True,
)

2021-05-04 23:37:08, INFO: Writing Network as Shapefile
2021-05-04 23:37:08, INFO: Writing Network as Shapefile
2021-05-04 23:37:08, INFO: Renaming DBF Node Variables
2021-05-04 23:37:08, INFO: Renaming DBF Node Variables
2021-05-04 23:37:08, INFO: Renaming variables so that they are DBF-safe
2021-05-04 23:37:08, INFO: Renaming variables so that they are DBF-safe
2021-05-04 23:37:20, INFO: Renaming DBF Link Variables
2021-05-04 23:37:20, INFO: Renaming DBF Link Variables
2021-05-04 23:37:20, INFO: Renaming variables so that they are DBF-safe
2021-05-04 23:37:20, INFO: Renaming variables so that they are DBF-safe
2021-05-04 23:37:24, INFO: Writing Node Shapes:
 - D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_09\nodes.shp
2021-05-04 23:37:24, INFO: Writing Node Shapes:
 - D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_09\nodes.shp
2021-05-04 23:38:09, INFO: Writing Link Shapes:
 - D:/github/lfs-cleaning/travel-model-two-networks\data\pro

In [97]:
model_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'links.txt'),
    output_node_txt = os.path.join(output_dir, 'nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
    #drive_only = True
)

2021-05-05 10:37:51, INFO: Starting fixed width conversion
2021-05-05 10:37:51, INFO: Starting fixed width conversion
2021-05-05 11:03:22, INFO: Writing out link database
2021-05-05 11:03:22, INFO: Writing out link database
2021-05-05 11:03:37, INFO: Writing out link header and width ----
2021-05-05 11:03:37, INFO: Writing out link header and width ----
2021-05-05 11:03:38, INFO: Starting fixed width conversion
2021-05-05 11:03:38, INFO: Starting fixed width conversion
2021-05-05 11:05:42, INFO: Writing out node database
2021-05-05 11:05:42, INFO: Writing out node database
2021-05-05 11:05:44, INFO: Writing out node header and width
2021-05-05 11:05:44, INFO: Writing out node header and width


In [98]:
%%capture
os.chdir(output_dir)
!make_cube.bat

links.txt(976000): 3085842;3032649;0;1;0; Contra Costa;   0.05330874299132302
links.txt(977000): 3026839;3066719;1;1;0; Contra Costa;  0.048891590645526714
links.txt(978000): 5005807;5009948;1;1;0;        Marin;   0.06645771477398354
links.txt(979000): 1547444;1522955;0;1;0;    San Mateo;   0.03858276320264887
links.txt(980000): 3541966;3517615;0;1;0;       Solano;  0.028451368419345358
links.txt(981000): 4539760;4501754;0;1;0;       Sonoma;    0.0534897372965782
links.txt(982000): 3524661;3532035;1;1;0;       Solano;   0.01840697689378509
links.txt(983000): 5020277;5016708;0;1;0;        Marin;  0.025322406118651445
links.txt(984000): 2563790;2548886;0;1;0;      Alameda;   0.07087576394435982
links.txt(985000): 2024762;2012580;0;1;0;  Santa Clara;  0.050700779409235716
links.txt(986000): 2013691;2155037;0;0;0;  Santa Clara;  0.008861854812808842
links.txt(987000): 4006351;4001221;1;1;0;         Napa;   0.03509878899956671
links.txt(988000): 1520771;1530036;0;1;0;    San Mateo;   0.0657

links.txt(1692000): 1021456;  12202;1;1;0;San Francisco;   0.0770100875507122
links.txt(1693000): 1022549;  15396;1;1;0;San Francisco;   0.0654311537179020
links.txt(1694000): 1023756;  10369;1;1;0;San Francisco;  0.06108124399744458
links.txt(1695000): 1025243;  16708;1;1;0;San Francisco;  0.04058270365168669
links.txt(1696000): 1026804;  14662;1;1;0;San Francisco;   0.0560322410815223
links.txt(1697000): 1500887; 112522;1;1;0;    San Mateo;   0.0416213770919007
links.txt(1698000): 1502790; 114634;1;0;0;    San Mateo;   0.0550567617416026
links.txt(1699000): 1504707; 111156;1;1;0;    San Mateo;   0.0752770829383756
links.txt(1700000): 1506472; 115525;1;1;0;    San Mateo;  0.05258826438146240
links.txt(1701000): 1508405; 117029;1;1;0;    San Mateo;   0.0503358814921022
links.txt(1702000): 1510395; 117978;1;1;0;    San Mateo;   0.0682802538464082
links.txt(1703000): 1512343; 112939;1;0;0;    San Mateo;   0.1629083057214681
links.txt(1704000): 1514211; 117531;1;1;0;    San Mateo;   0.085


NODEMERGE: 2166084

NODEMERGE: 2167084

NODEMERGE: 2168084

NODEMERGE: 2169084

NODEMERGE: 2170084

NODEMERGE: 2171084

NODEMERGE: 2172084

NODEMERGE: 2173084

NODEMERGE: 2174084

NODEMERGE: 2175084

NODEMERGE: 2176084

NODEMERGE: 2177084

NODEMERGE: 2178084

NODEMERGE: 2179084

NODEMERGE: 2180084

NODEMERGE: 2181084

NODEMERGE: 2182084

NODEMERGE: 2183084

NODEMERGE: 2184084

NODEMERGE: 2185084

NODEMERGE: 2186084

NODEMERGE: 2187084

NODEMERGE: 2188084

NODEMERGE: 2189084

NODEMERGE: 2190084

NODEMERGE: 2191084

NODEMERGE: 2192084

NODEMERGE: 2500138

NODEMERGE: 2501138

NODEMERGE: 2502138

NODEMERGE: 2503138

NODEMERGE: 2504138

NODEMERGE: 2505138

NODEMERGE: 2506138

NODEMERGE: 2507138

NODEMERGE: 2508138

NODEMERGE: 2509138

NODEMERGE: 2510138

NODEMERGE: 2511138

NODEMERGE: 2512138

NODEMERGE: 2513138

NODEMERGE: 2514138

NODEMERGE: 2515138

NODEMERGE: 2516138

NODEMERGE: 2517138

NODEMERGE: 2518138

NODEMERGE: 2519138

NODEMERGE: 2520138

NODEMERGE: 2521138

NODEMERGE: 2522138


LINKMERGE: 2001745-2009848     

LINKMERGE: 2002041-2029200     

LINKMERGE: 2002343-2153303     

LINKMERGE: 2002645-2142808     

LINKMERGE: 2002949-213491     

LINKMERGE: 2003247-2093747     

LINKMERGE: 2003540-2152007     

LINKMERGE: 2003834-2031822     

LINKMERGE: 2004129-2011414     

LINKMERGE: 2004426-2170583     

LINKMERGE: 2004724-2013791     

LINKMERGE: 2005018-2146107     

LINKMERGE: 2005316-215029     

LINKMERGE: 2005604-2171764     

LINKMERGE: 2005900-2107310     

LINKMERGE: 2006201-2016424     

LINKMERGE: 2006492-2056695     

LINKMERGE: 2006787-2114301     

LINKMERGE: 2007078-2008978     

LINKMERGE: 2007370-2172721     

LINKMERGE: 2007661-2086209     

LINKMERGE: 2007963-2143068     

LINKMERGE: 2008262-2065245     

LINKMERGE: 2008550-2156344     

LINKMERGE: 2008850-2109638     

LINKMERGE: 2009134-2172058     

LINKMERGE: 2009427-2064565     

LINKMERGE: 2009724-2013984     

LINKMERGE: 2010020-2058745     

LINKMERGE: 2010323-2159022     

LINKMERGE: 2

LINKMERGE: 3076148-3014021     

LINKMERGE: 3076496-413515     

LINKMERGE: 3076832-3078470     

LINKMERGE: 3077177-3019109     

LINKMERGE: 3077522-3064380     

LINKMERGE: 3077879-3060917     

LINKMERGE: 3078220-3061251     

LINKMERGE: 3078582-3017276     

LINKMERGE: 3078933-3016026     

LINKMERGE: 3079270-3066142     

LINKMERGE: 3079621-3011007     

LINKMERGE: 3079981-3059251     

LINKMERGE: 3080347-3076268     

LINKMERGE: 3080701-3034383     

LINKMERGE: 3081105-3052848     

LINKMERGE: 3081790-3005413     

LINKMERGE: 3082447-424182     

LINKMERGE: 3083098-3014646     

LINKMERGE: 3083760-416989     

LINKMERGE: 3084443-3023836     

LINKMERGE: 3085122-3027940     

LINKMERGE: 3085811-3004875     

LINKMERGE: 3086490-3036622     

LINKMERGE: 3087192-410843     

LINKMERGE: 3087870-3044569     

LINKMERGE: 3088581-3048457     

LINKMERGE: 3089271-3051988     

LINKMERGE: 3089973-3021493     

LINKMERGE: 3090673-410066     

LINKMERGE: 3091369-3061982     

LINKMERGE: 3092

## As transit line file (for CUBE)

In [89]:
mtc.write_cube_fare_files(
    roadway_network=step5_scenario.road_net,
    transit_network=step5_scenario.transit_net,
    parameters=parameters,
    outpath = os.path.join(output_dir)
)

In [91]:
standard_transit_net = StandardTransit.fromTransitNetwork(step5_scenario.transit_net, parameters = parameters)

2021-05-05 09:11:17, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-05-05 09:11:17, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-05-05 09:11:17, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2021-05-05 09:11:17, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


In [95]:
for agency in standard_transit_net.feed.routes.agency_raw_name.unique():
    sub_transit_net = copy.deepcopy(standard_transit_net)
    sub_transit_net.feed.trips = sub_transit_net.feed.trips[sub_transit_net.feed.trips.agency_raw_name == agency]
    mtc.write_as_cube_lin(sub_transit_net, parameters, outpath = os.path.join(output_dir, agency + "_transit.lin"))

2021-05-05 09:13:35, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-05-05 09:13:35, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-05-05 09:13:47, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-05-05 09:13:47, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-05-05 09:13:48, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-05-05 09:13:48, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-05-05 09:14:28, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-05-05 09:14:28, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-05-05 09:14:33, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-05-05 09:14:33, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-05-05 09:14:37, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-05-05 09:14:37, INFO: Converting GTFS Standard Pr

In [96]:
mtc.write_as_cube_lin(standard_transit_net, parameters, outpath = os.path.join(output_dir, "transit.lin"))

2021-05-05 09:26:10, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2021-05-05 09:26:10, INFO: Converting GTFS Standard Properties to MTC's Cube Standard


## As pickle

In [125]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_09.pickle')
pickle.dump(step5_scenario, open(working_scenario_filename, 'wb'))

## As standard network

In [99]:
step5_scenario.road_net.write(path = output_dir, filename = "v09")
step5_scenario.transit_net.write(path = output_dir, filename = "v09")

2021-05-10 12:53:22, INFO: Writing transit to directory: D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_09
2021-05-10 12:53:22, INFO: Writing transit to directory: D:/github/lfs-cleaning/travel-model-two-networks\data\processed\version_09


In [123]:
# save out link feather
# to_feather() does not like mixed type

links_feather = step5_scenario.road_net.links_df.copy()
links_feather.drop("geometry", axis = 1, inplace = True)

for c in links_feather.columns:
    if c in parameters.int_col:
        links_feather[c] = links_feather[c].astype(int)
    else:
        links_feather[c] = links_feather[c].astype(str)
        
links_feather.to_feather(path = os.path.join(output_dir, "v09_links.feather"))

c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\ipykernel_launcher.py:13: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  del sys.path[0]
